In [1]:
import os
from pathlib import Path
current_directory = Path().resolve()
os.chdir(Path(current_directory).resolve().parent.parent)

import pandas as pd
import matplotlib.pyplot as plt
import scienceplots
import chardet

plt.style.use(['science', 'no-latex', 'notebook', 'grid'])
plt.rcParams['text.usetex'] = False

file_path = current_directory / "r_2.csv"

if not file_path.exists():
    raise Exception(f"File {file_path} does not exist.")

with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())
file_encoding = result['encoding']

df = pd.read_csv(file_path, encoding=file_encoding, sep=',')

df.head()

,3,5,10,temperature
0,3.324407e-21,1.071195e-22,1.948996e-29,32.18783
1,4.330634e-21,1.394900e-22,2.680175e-29,34.05931
2,5.641424e-21,1.816426e-22,3.685660e-29,35.93079
3,7.348962e-21,2.365334e-22,5.068360e-29,37.80227
4,9.573335e-21,3.080116e-22,6.969790e-29,39.67375


In [2]:
import numpy as np
import scipy.stats as stats
from scipy.constants import R
from src.core.app_settings import NUC_MODELS_TABLE, NUC_MODELS_LIST

ALPHA_MIN = 0.005 # Минимальная конверсия
ALPHA_MAX = 0.995 # Максимальная конверсия
df["3_conversion"] = df["3"].cumsum() / df["3"].cumsum().max()

df["temperature"] = df["temperature"] + 273.15

def calculate_direct_diff_lhs(da_dT, f_a_val):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.log(da_dT / f_a_val)
        result[~np.isfinite(result)] = np.inf  # Заменить NaN и inf на бесконечность
    return result

def calculate_direct_diff_params(slope, intercept, beta):
    Ea = -slope * R
    A = np.exp(intercept) * int(beta)
    return Ea, A

def process_model(temperature, conversion, model_key, beta):
    da_dT = conversion.diff()
    model_func = NUC_MODELS_TABLE[model_key]["differential_form"]
    f_a_val = model_func(1 - conversion)

    lhs = calculate_direct_diff_lhs(da_dT, f_a_val)

    reverse_temperature = 1 / temperature
    valid_mask = np.isfinite(lhs) & np.isfinite(reverse_temperature)  # Отбираем только валидные данные
    
    if np.sum(valid_mask) < len(temperature) * 0.8:  # Если недостаточно данных для линейной регрессии
        return pd.DataFrame({
            'Model': [model_key],
            'R2_score': [np.nan],
            'Ea': [np.nan],
            'A': [np.nan],
        })
    
    # Линейная регрессия только с валидными значениями
    slope, intercept, r_value, p_value, std_err = stats.linregress(reverse_temperature[valid_mask], lhs[valid_mask])
    Ea, A = calculate_direct_diff_params(slope, intercept, beta)
    
    return pd.DataFrame({
        'Model': [model_key],
        'R2_score': [r_value**2],
        'Ea': [Ea],
        'A': [A],
    })

def trim_conversion(temperature, conversion):
    valid_mask = (conversion >= ALPHA_MIN) & (conversion <= ALPHA_MAX)
    trimmed_conversion = conversion[valid_mask]
    trimmed_temperature = temperature[valid_mask]
    
    return trimmed_temperature, trimmed_conversion

def obtain_direct_diff_coeffs(temperature, conversion, beta):
    direct_diff = pd.DataFrame(columns=['Model', 'R2_score', 'Ea', 'A'])
    trimmed_temperature, trimmed_conversion = trim_conversion(temperature, conversion)
    for i, model in enumerate(NUC_MODELS_LIST):
        temp_df = process_model(trimmed_temperature, trimmed_conversion, NUC_MODELS_LIST[i], beta)
        direct_diff = pd.concat([direct_diff, temp_df], ignore_index=True)

    return direct_diff

beta = 3
direct_diff = obtain_direct_diff_coeffs(df["temperature"], df["3_conversion"], beta)

df_sorted = direct_diff.sort_values(by='R2_score', ascending=False)

df_sorted['R2_score'] = df_sorted['R2_score'].round(4)
df_sorted['Ea'] = df_sorted['Ea'].round()
df_sorted['A'] = df_sorted['A'].apply(lambda x: f"{x:.3e}")

df_sorted

C:\Users\davjd\AppData\Local\Temp\ipykernel_15504\2278140823.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  direct_diff = pd.concat([direct_diff, temp_df], ignore_index=True)


,Model,R2_score,Ea,A
1,A2/3,0.9969,556954.0,1.783e+45
4,A3/4,0.9965,494721.0,1.381e+40
30,G6,0.9949,-749946.0,2.648e-61
20,F1/A1,0.9948,370254.0,7.869e+29
24,F3/4,0.9915,324396.0,2.486e+25
12,D5,0.9901,837927.0,4.112e+67
29,G5,0.9898,-376546.0,4.532e-31
3,A3/2,0.9895,245788.0,3.985e+19
32,G8,0.9881,654494.0,4.722e+52
10,D3,0.9881,654494.0,1.049e+52
